In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from scipy import stats


In [3]:
df = pd.read_csv('../../data/sweep_0_10_data.csv')
# df2 = pd.read_csv('../../data/sweep_0_10_data2.csv')
# df = pd.concat([df1, df2], axis=1)
df.drop(columns=["run_id", "run_name", "_runtime", "_step", "_timestamp"], inplace=True)

In [4]:
df.tail()

,amd,rcm,padding,matrix_name,partial_gauss,n,rank,max_mode_size,z,tile_size
19959,True,False,0,ex3,0,1821.0,9.0,607.0,52685.0,607.0
19960,True,False,0,ex3,0,1821.0,1.0,1821.0,52685.0,1821.0
19961,True,True,0,ex3,0,1821.0,38697.0,607.0,52685.0,3.0
19962,True,True,0,ex3,0,1821.0,9.0,607.0,52685.0,607.0
19963,True,True,0,ex3,0,1821.0,1.0,1821.0,52685.0,1821.0


In [5]:
df["log_obj_func"] = np.log(df["max_mode_size"]**6 + df["max_mode_size"]**3 *  df["rank"] + df["max_mode_size"]**2 *  df["rank"]**2)
df["obj_func"] = np.exp(df["log_obj_func"])

In [6]:
baseline_df = df[(df['amd'] == False) & (df['rcm'] == False) & (df['partial_gauss'] == 0) & (df['padding'] == 0)]
padding_df = df[(df['amd'] == False) & (df['rcm'] == False) & (df['partial_gauss'] == 0)]
amd_df = df[(df['rcm'] == False) & (df['partial_gauss'] == 0) & (df['padding'] == 0)]
rcm_df = df[(df['amd'] == False) & (df['partial_gauss'] == 0) & (df['padding'] == 0)]
partial_gauss_df = df[(df['amd'] == False) & (df['rcm'] == False) & (df['padding'] == 0)]

In [7]:
df.head()

,amd,rcm,padding,matrix_name,partial_gauss,n,rank,max_mode_size,z,tile_size,log_obj_func,obj_func
0,False,False,10,ex15,10,6867.0,33252.0,109.0,98623.0,3.0,30.329488,1.485693e+13
1,False,False,10,ex15,10,6867.0,8766.0,109.0,98623.0,7.0,28.587079,2.601421e+12
2,False,False,10,ex15,10,6867.0,6816.0,109.0,98623.0,9.0,28.436556,2.237893e+12
3,False,False,10,ex15,10,6867.0,2921.0,109.0,98623.0,21.0,28.208900,1.782254e+12
4,False,False,10,ex15,10,6867.0,539.0,109.0,98623.0,63.0,28.150559,1.681250e+12


In [9]:
# Tile size choice influence on TT-MALS runtime
fig = px.line(baseline_df, x="tile_size", y="log_obj_func", color='matrix_name', symbol="matrix_name", markers=True, log_x=True,
              labels={
                     "tile_size": "Tile size",
                     "matrix_name": "Matrix name",
                 }
              )
fig.update_layout(
    title={
        'text': "Influence of tile size choice on TT-MALS runtime",
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    plot_bgcolor='white',   # Plot area background color
    paper_bgcolor='white',  # Entire figure background color
    font=dict(color='black'), # Font color
    yaxis_title=r'$\log(I^6 + rI^3 + r^2I^2)$'
)
fig.show()
fig.write_image("plots/baseline_tile_size_vs_log_obj_func.pdf")

In [11]:
# trade-off between rank and max mode size - baseline
fig = px.line(baseline_df, x="rank", y="max_mode_size", color="matrix_name", symbol="matrix_name", log_x=True, log_y=True, 
                 labels={
                     "rank": "Rank (r)",
                     "matrix_name": "Matrix name",
                     "max_mode_size": "Maximum mode size (I)",
                 })
fig.update_layout(
    title={
        'text': "Trade-off between rank and maximum mode size",
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    plot_bgcolor='white',   # Plot area background color
    paper_bgcolor='white',  # Entire figure background color
    font=dict(color='black') # Font color
)
fig.show()

fig.write_image("plots/baseline_max_mode_size_vs_rank.pdf")

In [21]:
# explore effects of padding: 
def line_plot_padding_tile_size_tt_mals_runtime(matrix_str: str):
    default_colorscale = px.colors.sequential.Jet
    colors = px.colors.sample_colorscale(default_colorscale, 11)
    fig = px.line(padding_df[padding_df["matrix_name"] == matrix_str], x="tile_size", y="log_obj_func", color="padding", symbol="padding", log_x=True, color_discrete_sequence=colors,
                     labels={
                         "tile_size": "Tile size",
                     })
    fig.update_layout(
        title={
            'text': "Influence of tile size choice and padding on TT-MALS runtime ({})".format(matrix_str),
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'
        },
        plot_bgcolor='white',   # Plot area background color
        paper_bgcolor='white',  # Entire figure background color
        font=dict(color='black'), # Font color
        yaxis_title=r'$\log(I^6 + rI^3 + r^2I^2)$'
    )
    fig.show()
    
    fig.write_image("plots/{}_padding_tile_size_vs_log_obj_func.pdf".format(matrix_str))

In [22]:
line_plot_padding_tile_size_tt_mals_runtime("ex3")
line_plot_padding_tile_size_tt_mals_runtime("ex10hs")

In [100]:
fig = px.line(padding_df[padding_df["matrix_name"] == "ex10"], x="rank", y="max_mode_size", color="padding", symbol="matrix_name", log_x=True, log_y=True, 
                 labels={
                     "rank": "Maximum Rank (r)",
                     "matrix_name": "Matrix Name",
                     "max_mode_size": "Maximum Mode Size (I)",
                 })
fig.update_layout(
    title={
        'text': "Influence of tile size choice and RCM on TT-MALS runtime",
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    plot_bgcolor='white',   # Plot area background color
    paper_bgcolor='white',  # Entire figure background color
    font=dict(color='black'), # Font color
    yaxis_title=r'$\log(I^6 + rI^3 + r^2I^2)$'
)
fig.show()

fig.write_image("plots/rcm_max_mode_size_vs_max_rank.pdf")